In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
embeddings = np.load("model/sentence_tranformers.npy")
metadata = pd.read_csv('../data/data/processed_data/PDF_Engineering.csv') 


In [20]:
metadata.head()

,Filename,Extracted_Text,Category,Cleaned_Text
0,10030015.pdf,ENGINEERING LAB TECHNICIAN\nCareer Focus\nMy m...,Engineering,engineering lab technician career focus main o...
1,10219099.pdf,EQUIPMENT ENGINEERING TECHNICIAN\nProfessional...,Engineering,equipment engineering technician professional ...
2,10624813.pdf,ENGINEERING OPERATIONS DIRECTOR\nExecutive Pro...,Engineering,engineering operations director executive prof...
3,10712803.pdf,ENGINEERING SERVICES MANAGER\nExecutive Profil...,Engineering,engineering services manager executive profile...
4,10985403.pdf,MECHANICAL ENGINEERING INTERN\nSummary\nCAD | ...,Engineering,mechanical engineering intern summary cad cam ...


In [25]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

texts = metadata["Cleaned_Text"].tolist()
embeddings = model.encode(texts, show_progress_bar=True, normalize_embeddings=True)


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.34it/s]


In [29]:
jd_example = "We are looking for a Python Backend Engineer with experience in APIs, SQL databases, cloud deployment, Docker, and problem solving."
jd_embedding = model.encode([jd_example], normalize_embeddings=True)[0]


In [36]:
scores = cosine_similarity([jd_embedding], embeddings).flatten()
# simple keyword-based explanation for why a CV might fit the job description
keywords = ["python", "api", "apis", "sql", "docker", "cloud", "problem solving", "backend", "backend engineer"]
def explain_fit(text, keywords=keywords):
	text_l = str(text).lower()
	matches = [k for k in keywords if k in text_l]
	return ", ".join(matches) if matches else "No matching keywords"
# add explanation column
metadata["Why it fix?"] = metadata["Cleaned_Text"].apply(explain_fit)
top_idx = scores.argsort()[::-1][:10]
results = metadata.iloc[top_idx].copy()
results["score"] = scores[top_idx]
results.head()


,Filename,Extracted_Text,Category,Cleaned_Text,Why it fix?,score
2,10624813.pdf,ENGINEERING OPERATIONS DIRECTOR\nExecutive Pro...,Engineering,engineering operations director executive prof...,"python, api, sql",0.307248
1,10219099.pdf,EQUIPMENT ENGINEERING TECHNICIAN\nProfessional...,Engineering,equipment engineering technician professional ...,problem solving,0.282456
3,10712803.pdf,ENGINEERING SERVICES MANAGER\nExecutive Profil...,Engineering,engineering services manager executive profile...,No matching keywords,0.263330
0,10030015.pdf,ENGINEERING LAB TECHNICIAN\nCareer Focus\nMy m...,Engineering,engineering lab technician career focus main o...,No matching keywords,0.251930
4,10985403.pdf,MECHANICAL ENGINEERING INTERN\nSummary\nCAD | ...,Engineering,mechanical engineering intern summary cad cam ...,api,0.235576
